# 7. 챗봇API

## 7.1 chatbot API?

* 실습시 작성한 챗봇엔진서버와 직접통신을 하는 `카카오톡, 네이버톡톡등과 같은 메신저플랫폼과 챗봇엔진을 사용할 수 있는 챗봇API서버를 구축`해야 한다.
* 챗봇기능을 지원하는 메신저플랫폼과 통신하기 위해서는 `REST API방식으로 챗봇서버를 구현`해야 한다.

## 7.2 Python Flask

* Flask는 파이썬 기반의 경량화된 프레임워크
* `웹애플리케이션 경량 프레임워크 Flask'를 사용

### 7.2.1 Hello Flask

In [1]:
!mkdir .\hello_flask
!pip show flask

Name: Flask
Version: 2.2.5
Summary: A simple framework for building complex web applications.
Home-page: https://palletsprojects.com/p/flask
Author: Armin Ronacher
Author-email: armin.ronacher@active-4.com
License: BSD-3-Clause
Location: C:\Anaconda3\Lib\site-packages
Requires: click, itsdangerous, Jinja2, Werkzeug
Required-by: 


In [5]:
%%writefile ./hello_flask/app.py
# 1. Hello Flask 출력 & Login Form 출력
# 실행 : python d:\lec\05.python\hello_flask\app.py
from flask import Flask

app = Flask(__name__)

# https://localhost:5000/
@app.route('/')
def hello():
    return 'Hello Falsk'

# https://localhost:5000/login
@app.route('/login')
def login():
    return 'Login Form'

# https://localhost:5000/chatbot
@app.route('/chatbot')
def chatbot():
    return '<h1>주문처리완료 : 주문해주셔서 감사합니다!</h1>'

if __name__ == '__main__':
    app.run()

Overwriting ./hello_flask/app.py


In [15]:
%%writefile ./hello_flask/app_1.py
# 2. 동적으로 변수를 처리하는 방법
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def hello():
    return 'Hello Flask?'

# https://localhost:5000/info/홍길동
@app.route('/info/<name>')
def get_name(name):
    return f'Hello {name}'

# https://localhost:5000/JSON/1000/자장면 1개 주문할게요?
@app.route('/JSON/<int:id>/<message>')
def send_message(id, message):
    json = {
        "id": id,
        "message": message,
        "xxx": "====================>"
    }
    return json

# https://localhost:5000/손흥민/32
@app.route('/<name>/<int:age>')
def hello_user(name, age):
    return render_template('hello.html', name=name, age=age)

if __name__ == '__main__':
    app.run()

Overwriting ./hello_flask/app_1.py


In [13]:
!mkdir .\hello_flask\templates

In [16]:
%%writefile .\hello_flask\templates\hello.html
<h1>이름 : {{name}}</h1>
<h2>나이 : {{age}}</h2>

Overwriting .\hello_flask\templates\hello.html


### 7.2.2 기본적인 REST API 서비스 구현하기

In [8]:
!mkdir .\basic_restapi

In [25]:
%%writefile ./basic_restapi/app.py
# 3. 기본적인 REST API 서버 구현하기
# 실행 : python d:\lec\05.python\basic_restapi\app.py

from flask import Flask, request, jsonify

app = Flask(__name__)

# 서비스리소스
resource = []

# 1) 사용자정보조회
@app.route('/user/<int:userid>', methods=['GET'])
def get_user(userid):
    for user in resource:
        if user['userid'] is userid:
            return jsonify(user)
    
# 2) 사용자추가
@app.route('/user', methods=['POST'])
def add_user():
    user = request.get_json()
    resource.append(user)
    return jsonify(resource)

if __name__ == '__main__':
    app.run()

Overwriting ./basic_restapi/app.py


##### REST API서비스의 구동을 지원해 주는 Tool
1. Postman
2. vscode에서 Thunder Client
3. 크롬에서 Talend API Tester

## 7.3 챗봇API구현

##### 발화자질문
```json
{
  "query":"오늘 자장면 주문할게요"
}
```

##### 질의답변
```json
{
    "Answer": "자장면 주문 처리 완료되었습니다. \n주문해주셔서 감사합니다.",
    "AnswerImageUrl": null,
    "Intent": "주문",
    "NER": "[('오늘', 'B_DT'), ('자장면', 'B_FOOD'), ('주문', 'O')]",
    "Query": "오늘 자장면 주문할게요"
}
```


In [36]:
!mkdir .\chatbot_api
!mkdir .\chatbot_api\templates

하위 디렉터리 또는 파일 .\chatbot_api이(가) 이미 있습니다.
하위 디렉터리 또는 파일 .\chatbot_api\templates이(가) 이미 있습니다.


In [30]:
%%writefile ./chatbot_api/templates/index.html
<h1>Index.html</h1>
<h2>Chatbot REST API Home Page</h1>

Writing ./chabot_api/templates/index.html


In [37]:
%%writefile ./chatbot_api/app.py
# chatbot REST API 서버 구현하기
from flask import Flask, request, jsonify, abort, render_template
import socket
import json

# 챗봇엔진서버접속정보, ip, port
host = "127.0.0.1"  # chatbot server
port = 5050

# Flask객체
app = Flask(__name__)

# ./chabot_api/templates/index.html
@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

# 1) chatbot server와 통신
def get_answer_from_engine(bottype, query):
    # chatbot서버와 연결
    mySocket = socket.socket()
    mySocket.connect((host, port))

    # chatbot서버에 request(query)
    json_data = {
        "Query": query,
        "BotType": bottype
    }

    # chatbot서버에 query전송
    message = json.dumps(json_data)
    mySocket.send(message.encode())

    # chatbot서버에서 response
    data = mySocket.recv(2048).decode()
    ret_data = json.loads(data)

    # chatbot서버에 연결된 socket자원을 해제
    mySocket.close()

    return ret_data

# 2) chatbot엔진에 query를 전송하는 API
# http://localhost:5000/query/TEST
@app.route('/query/<bot_type>', methods=['POST'])
def query(bot_type):
   
    body = request.get_json()

    try:
        if bot_type == "TEST":
            # chatbot api test
            ret = get_answer_from_engine(bottype=bot_type, query=body['query'])
            return jsonify(ret)
        elif bot_type == "KAKAO": # 카카오톡 skill
            pass
        elif bot_type == "NAVER": # 네이버톡톡 Web Hook 처리
            pass
        else:
            # 정의되지 않은 bot_type일 경우 404발생
            abort(404)
    except Exception as e:
        # 예외발생시 500에러 발생
        abort(500)
            
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)  # WebServer

Overwriting ./chatbot_api/app.py


### 7.3.1 챗봇 REST API애플리케이션 시작

* cmd창(Termnal)
  1. 챗봇서버 시작    : root폴더 > python bot.py
  2. 챗봇WEB서버 시작 : root폴더 > python ./chatbot_api/app.py
  3. 챗봇WEB서버 구동 : REST API서비스의 구동을 지원 Tool : Talend API Tester
     - http://localhost:5000/query/TEST
       
```json
{   "query":"오늘 자장면 주문할게요"
  ```